# Subsetting an Unstructured Grid: Analysis Over Chicago

Authors: [Philip Chmielowiec](https://github.com/philipc2)

This usage example showcases various ways of subsetting an unstructured grid using UXarray, focussing on analyzing a region around Chicago, Illinois.

In [ ]:
import uxarray as ux
import geoviews.feature as gf
import cartopy.crs as ccrs
import holoviews as hv

import warnings

import geocat.datafiles as geodf


plot_opts = {"width" : 700, "height": 350}

hv.extension('bokeh')


warnings.filterwarnings("ignore")

## Setup

In [ ]:
datafiles = (
    geodf.get(
        "netcdf_files/MPAS/FalkoJudt/dyamond_1/30km/diag.2016-08-20_00.00.00_subset.nc"
    ),
    geodf.get("netcdf_files/MPAS/FalkoJudt/dyamond_1/30km/x1.655362.grid_subset.nc"),
)

In [ ]:
uxds = ux.open_dataset(datafiles[1], datafiles[0])

In [ ]:
clim = (uxds['relhum_200hPa'][0].values.min(), uxds['relhum_200hPa'][0].values.max())

In [ ]:
features = gf.coastline(projection=ccrs.PlateCarree(), line_width=1, scale='50m') * \
           gf.states(projection=ccrs.PlateCarree(), line_width=1, scale='50m')

## Global Grid

Many unstructured grids, such as those from global climate models, span the entire surface of a sphere (both with or without masks).

UXarray is built around supporting grids at a global extent, considering the spherical geometry that is involved (wrap around at the antimeridian, pole points, etc.)


In [ ]:
uxds['relhum_200hPa'][0].plot.rasterize(method='polygon',
                                        exclude_antimeridian=True,
                                        title = "Global Grid",
                                        **plot_opts) * features

In addition to plotting global grids, we can perform analysis operations.

In [ ]:
uxds['relhum_200hPa'][0].values.mean()

## Bounding Box

We can declare a bounding box centered about the Chicago area by specifying the minimum and maximum longitude and latitude bounds.

In [ ]:
lon_bounds = (-87.6298 - 2, -87.6298 + 2)
lat_bounds = (41.8781 - 2, 41.8781 + 2)

In [ ]:
bbox_subset_nodes = uxds['relhum_200hPa'][0].subset.bounding_box(lon_bounds, lat_bounds, element='nodes')

bbox_subset_nodes.plot.rasterize(method='polygon',
                                 exclude_antimeridian=True,
                                 clim=clim,
                                 title="Bounding Box Subset",
                                 **plot_opts) * features

In [ ]:
bbox_subset_faces= uxds['relhum_200hPa'][0].subset.bounding_box(lon_bounds, lat_bounds, element='face centers')

bbox_subset_faces.plot.rasterize(method='polygon',
                                 exclude_antimeridian=True,
                                 clim=clim,
                                 title="Bounding Box Subset (Face Center Query)",
                                 **plot_opts) * features

## Bounding Circle

In [ ]:
center_coord = [-87.6298, 41.8781]

r = 2

In [ ]:
bcircle_subset = uxds['relhum_200hPa'][0].subset.bounding_circle(center_coord, r)

bcircle_subset.plot.rasterize(method='polygon',
                              exclude_antimeridian=True,
                              clim=clim,
                              title="Bounding Circle Subset",
                              **plot_opts) * features

## Nearest Neighbor

In [ ]:
center_coord = [-87.6298, 41.8781]

In [ ]:
nn_subset = uxds['relhum_200hPa'][0].subset.nearest_neighbor(center_coord, k = 30, tree_type='nodes')

nn_subset.plot.rasterize(method='polygon', exclude_antimeridian=True,
                         clim=clim,
                         title= "Nearest Neighbor Subset",
                         **plot_opts) * features

In [ ]:
nn_subset_120 = uxds['relhum_200hPa'][0].subset.nearest_neighbor(center_coord, k = 120, tree_type="face centers")

nn_subset_120.plot.rasterize(method='polygon',
                         exclude_antimeridian=True,
                         clim=clim,
                        title= "Nearest Neighbor Subset (120 Faces)",
                         **plot_opts) * features

In [ ]:
nn_subset_1 = uxds['relhum_200hPa'][0].subset.nearest_neighbor(center_coord, k = 1, tree_type="face centers")

nn_subset_1.plot.rasterize(method='polygon', exclude_antimeridian=True,
                         clim=clim,
                         title= "Nearest Neighbor Subset (Closest Face)",
                         **plot_opts) * features

## Analysis Operators

Since each subset is a newly initialized grid, analysis operators can be directly performed without needing to specify anything else.

In [ ]:
bbox_subset_nodes.values.mean(), bbox_subset_faces.values.mean(), bcircle_subset.values.mean()

In [ ]:
bbox_subset_nodes.values.std(), bbox_subset_faces.values.std(), bcircle_subset.values.std()

In [ ]:
bbox_subset_nodes.values.min(), bbox_subset_faces.values.min(), bcircle_subset.values.min()

In [ ]:
bbox_subset_nodes.values.max(), bbox_subset_faces.values.max(), bcircle_subset.values.max()